In [1]:
import os
import logging # doesn't work well in a notebook, without some fu
import rmgpy
import rmgpy.rmg
import rmgpy.rmg.input
#from rmgpy.display import display
from IPython.display import display
from rmgpy.chemkin import loadChemkinFile, readSpeciesBlock, readThermoBlock, readReactionsBlock, removeCommentFromLine
from rmgpy.data.thermo import Entry, saveEntry
from rmgpy.data.base import Entry as kinEntry
from rmgpy.data.kinetics.common import saveEntry as kinSaveEntry
from rmgpy.molecule import Molecule
from rmgpy.rmg.model import Species
from rmgpy.reaction import Reaction
from rmgpy.kinetics import Arrhenius
from rmgpy.cantherm.output import prettify
from rmgpy.rmg.main import RMG, initializeLog
from rmgpy.molecule.draw import MoleculeDrawer
import cPickle as pickle
import pandas as pd



In [2]:
with open("importerKinetics.pkl", "r") as f:
    importerKinetics = pickle.load(f)

In [3]:
importerKinetics

{Reaction(reactants=[Species(label="OH*", molecule=[Molecule(SMILES="[OH]")]), Species(label="O", molecule=[Molecule(SMILES="[O]")])], products=[Species(label="OH", molecule=[Molecule(SMILES="[OH]")]), Species(label="O", molecule=[Molecule(SMILES="[O]")])]): {'PCI2013/527-Sheen': Arrhenius(A=(1.5e+12,'cm^3/(mol*s)'), n=0.5, Ea=(0,'cal/mol'), T0=(1,'K'))},
 Reaction(reactants=[Species(label="nc4h9oh", molecule=[Molecule(SMILES="CCCCO")]), Species(label="oh", molecule=[Molecule(SMILES="[OH]")])], products=[Species(label="c4h8oh-4", molecule=[Molecule(SMILES="[CH2]CCCO")]), Species(label="h2o", molecule=[Molecule(SMILES="O")])]): {'CombFlame2012/2028-Sarathy': Arrhenius(A=(5.28e+09,'cm^3/(mol*s)'), n=0.97, Ea=(1586,'cal/mol'), T0=(1,'K')),
  'CombFlame2013/1939-Cai': Arrhenius(A=(5.28e+09,'cm^3/(mol*s)'), n=0.97, Ea=(1586,'cal/mol'), T0=(1,'K')),
  'CombFlame2014/657-Jin': Arrhenius(A=(5.28e+09,'cm^3/(mol*s)'), n=0.97, Ea=(1586,'cal/mol'), T0=(1,'K')),
  'PCI2013/289-Dagaut': Arrhenius(A=

In [4]:
i = 0
comparisonKinetics = {}
for rxn in importerKinetics:
    
    
    mols = []
    for reactant in rxn.reactants:
        mols.append(reactant.molecule[0])
    for product in rxn.products:
        mols.append(product.molecule[0])

    if Molecule().fromSMILES("OO") and Molecule().fromSMILES("[O]O") in mols and len(mols) == 4:
        print "Reaction WAS OOH abstraction"
        print rxn
        print
        comparisonKinetics[rxn] = importerKinetics[rxn]
        i += 1
    elif len(mols) != 4:
        print "Reaction was not bimolecular"
        print rxn
        print
        
    else:
        print "Reaction was not OOH Abstraction"
        print rxn
        print 

Reaction was not OOH Abstraction
OH* + O <=> OH + O

Reaction was not OOH Abstraction
nc4h9oh + oh <=> c4h8oh-4 + h2o

Reaction was not OOH Abstraction
IC4H9P + N1C4H9OH => IC4H10 + CH2CH2CH2CH2OH

Reaction was not OOH Abstraction
c4h8-2 + sc4h9o2 <=> c4h71-3 + sc4h9o2h

Reaction was not OOH Abstraction
nC4H7 + CH3 <=> C4H6 + CH4

Reaction was not OOH Abstraction
ic4h8 + c3h5-a <=> ic4h7 + c3h6

Reaction was not OOH Abstraction
IC4H9T + CH3COCH3 => IC4H10 + CH3COCH2

Reaction was not OOH Abstraction
O + C2H5 <=> OH + C2H4

Reaction WAS OOH abstraction
c4h8-2 + ho2 <=> c4h8o2-3 + oh

Reaction was not OOH Abstraction
ic4h7o + oh <=> ic3h5cho + h2o

Reaction was not OOH Abstraction
c4h8-2 + ch3o2 <=> c4h8o2-3 + ch3o

Reaction was not OOH Abstraction
ch3oh + sc4h9o2 <=> ch2oh + sc4h9o2h

Reaction was not OOH Abstraction
CH2CHCHO + aC3H5 <=> C3H6 + C2H3CO

Reaction was not bimolecular
c2h2 + c2h5 <=> c4h71-1

Reaction was not bimolecular
c3h4-a + ch3 <=> c4h71-2

Reaction was not bimolecula

In [9]:
with open("comparisonOOHRxns.py", "r") as f:
    lines = f.readlines()
rmgRxn = eval(lines[0].split('\n')[0])

split = rmgRxn.toChemkin(kinetics=False).split()
reactants = split[0], split[2]

a, b = rmgRxn.reactants
a


In [5]:
print "There were {} total imported reactions and {} bimolecular OOH reactions.".format(len(importerKinetics), len(comparisonKinetics))

There were 6303 total imported reactions and 464 bimolecular OOH reactions.


In [6]:
weight_dict = {}
for item in comparisonKinetics.keys():
    
    try:
        weight = item.reactants[0].molecule[0].getMolecularWeight() + item.reactants[1].molecule[0].getMolecularWeight()
    except Exception as err:
        print str(item) + ': ' + str(err)
        
    weight_dict[item] = weight

pre_sorted = pd.DataFrame(weight_dict, index=[0]).transpose()
sorted_dict = pre_sorted.sort_values(0).transpose()
with open("comparisonOOHRxns.py", "w") as q:
    for rxn in sorted_dict.columns:
        q.write(rxn.__repr__() + '\n')



IC4T-OOQOOH => CH2O + CH3COCH3 + HO2: list index out of range


# This next portion is designed to find reactions that have already been run. tbh, this portion is kinda pointless cuz autoTST already looks for previously run reactions

```
nateDirectory = "/gss_gpfs_scratch/harms.n/QMscratch"
westGroupDirectory = "/gss_gpfs_scratch/westgroup/QMscratch"
dataDumpDirectory = "/gss_gpfs_scratch/westgroup/ScienceDataDump/Reactions/H_Abstraction"

nateDirectory = "/Users/nathan/Code/scratch_test/nate/"
westGroupDirectory = "/Users/nathan/Code/scratch_test/west/"
dataDumpDirectory = "/Users/nathan/Code/scratch_test/dump/"

nate_reactions = os.listdir(nateDirectory)
west_reactions = os.listdir(westGroupDirectory)
dump_reactions = os.listdir(dataDumpDirectory)

previous_reactions = []
paths = []
for reaction in nate_reactions:
    if reaction not in previous_reactions:
        previous_reactions.append(reaction)
        paths.append(os.path.join(nateDirectory, reaction, "output.py"))
    else:
        print "Duplicate reaction {}".format(reaction)
        
for reaction in west_reactions:
    if reaction not in previous_reactions:
        previous_reactions.append(reaction)
        paths.append(os.path.join(westGroupDirectory, reaction, "output.py"))
    else:
        print "Duplicate reaction {}".format(reaction)
        
for reaction in dump_reactions:
    if reaction not in previous_reactions:
        previous_reactions.append(reaction)
        try:
            paths.append(os.path.join(dataDumpDirectory, reaction, "m062x", "output.py"))
        except:
            print "Directory error"
    else:
        print "Duplicate reaction {}".format(reaction)
        
```
        

```
previous_reactions, paths

to_try = []
fail = 0
for path in paths:
    print path
    try:
        assert os.path.exists(path), "Doesn't have output.py file"
        to_try.append(path)
        
    except Exception as err:
        print "# ⚠️", path, "Gave error", str(err)
len(to_try)
    
for path in to_try:
    print path
    
    try:
        with open(path) as f:
            print "opened"
            lines = f.readlines()
        start = lines.index('kinetics(\n')
        print lines[start:]
        
        with 
    except Exception as err:
        print "# ⚠️", path, "Gave error", str(err)
        fail += 1
```    



```
## Using the reaciton filtering tool used in filter kinetics

to_try = []
fail = 0
success = 0
for path in paths:
    print path
    try:
        assert os.path.exists(path), "Doesn't have output.py file"
        to_try.append(path)
        
    except Exception as err:
        print "# ⚠️", reaction, "Gave error", str(err)
len(to_try)

for path in to_try:
    try:
        print path
        
        with open(path) as f:
            lines = f.readlines()
        start = lines.index('kinetics(\n')
        print
        print "# {} from {}".format(path)
        print ''.join(lines[start:])
        success += 1
        
    except Exception as err:
        print "# ⚠️", path, "Gave error", str(err)
        fail += 1
        
to_try
```

```
%%capture cap --no-stderr
for path in to_try:
    try:
        print path
        
        with open(path) as f:
            lines = f.readlines()
        start = lines.index('kinetics(\n')
        print
        print "# {} from {}".format(path)
        print ''.join(lines[start:])
        success += 1
        
    except Exception as err:
        print "# ⚠️", reaction, "Gave error", str(err)
        fail += 1
```